# 2. Data Preparation (Transform Raw JSON to Tables) <a id="data-preparation"></a>

## 2.1 Create `prizes.csv` (Prize-level table) <a id="create-prizes"></a>

**Goal:** Build a clean prize-level table with **one row per Nobel Prize (award year × category)**.

**Inputs:**
- `nobel_prizes` (loaded from the latest `data/raw/nobelPrizes_*.json`)

**Output file:**
- `data/processed/prizes.csv`

**Fields extracted (core columns):**
- `prize_id` (stable key for joins)
- `award_year`
- `category`
- `category_full`
- `date_awarded`
- `prize_status`
- `prize_amount`
- `prize_amount_adjusted`

**Actions in the next code cell:**
- Extract required fields from each prize JSON record
- Normalize text fields (English values)
- Create `prize_id = "{award_year}_{category}"` (normalized)
- Convert `award_year` to numeric
- Remove duplicates and sort
- Save as CSV

**Validation / checks:**
- `prizes.shape`, `prizes.dtypes`
- `prizes["award_year"].max()` should be **2025**


In [1]:
# Step 1.1 — Imports + project paths (notebook is inside /notebooks)

from pathlib import Path
import json
import time
from datetime import datetime, timezone

import numpy as np
import pandas as pd
import requests

# --- Resolve project root ---
# notebook is in: Data_Analytics_Project/notebooks/
# then project root is one folder up from current working directory
CWD = Path.cwd().resolve()
PROJECT_ROOT = CWD.parent

# Expected structure check
print("CWD:", CWD)
print("PROJECT_ROOT:", PROJECT_ROOT)
print("Has /notebooks:", (PROJECT_ROOT / "notebooks").is_dir())
print("Has /data     :", (PROJECT_ROOT / "data").is_dir())

# --- Define data folders (outside notebooks) ---
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"

RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("\nFolders ready:")
print("RAW_DIR      :", RAW_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)


CWD: C:\0_DA\Source\Data_Analytics_Project\notebooks
PROJECT_ROOT: C:\0_DA\Source\Data_Analytics_Project
Has /notebooks: True
Has /data     : True

Folders ready:
RAW_DIR      : C:\0_DA\Source\Data_Analytics_Project\data\raw
PROCESSED_DIR: C:\0_DA\Source\Data_Analytics_Project\data\processed


In [2]:
# Step 1.4 — Load latest raw JSON snapshots

import json

# latest files (sorted by name)
prizes_file = sorted(RAW_DIR.glob("nobelPrizes_*.json"))[-1]
laureates_file = sorted(RAW_DIR.glob("laureates_*.json"))[-1]

# load JSON
with open(prizes_file, "r", encoding="utf-8") as f:
    nobel_prizes = json.load(f)

with open(laureates_file, "r", encoding="utf-8") as f:
    laureates = json.load(f)

print("Loaded:")
print(" -", prizes_file, "records:", len(nobel_prizes))
print(" -", laureates_file, "records:", len(laureates))


Loaded:
 - C:\0_DA\Source\Data_Analytics_Project\data\raw\nobelPrizes_20260206T113505Z.json records: 682
 - C:\0_DA\Source\Data_Analytics_Project\data\raw\laureates_20260206T113505Z.json records: 1018


In [4]:
# Step 2.1 — Create prizes.csv

def pick_en(x):
    """Return English text from dict like {'en': 'Physics'}; otherwise return x."""
    if isinstance(x, dict):
        return x.get("en") or x.get("se") or next(iter(x.values()), None)
    return x

rows = []

for p in nobel_prizes:
    award_year = p.get("awardYear")
    award_year_num = pd.to_numeric(award_year, errors="coerce")

    category = pick_en(p.get("category"))
    category_full = pick_en(p.get("categoryFullName"))

    prize_amount = p.get("prizeAmount")
    prize_amount_adj = p.get("prizeAmountAdjusted")
    date_awarded = p.get("dateAwarded")
    prize_status = p.get("prizeStatus")

    # stable key for later joins (year + category)
    prize_id = None
    if award_year and category:
        prize_id = f"{award_year}_{str(category).strip().lower().replace(' ', '_')}"

    rows.append({
        "prize_id": prize_id,
        "award_year": award_year_num,
        "category": category,
        "category_full": category_full,
        "date_awarded": date_awarded,
        "prize_status": prize_status,
        "prize_amount": prize_amount,
        "prize_amount_adjusted": prize_amount_adj,
    })

prizes = (
    pd.DataFrame(rows)
    .dropna(subset=["prize_id"])
    .drop_duplicates(subset=["prize_id"])
    .sort_values(["award_year", "category"])
    .reset_index(drop=True)
)

# Save to processed
prizes_csv_path = PROCESSED_DIR / "prizes.csv"
prizes.to_csv(prizes_csv_path, index=False)

print("✅ Saved:", prizes_csv_path)
print("Shape:", prizes.shape)
prizes.head(5)
prizes.dtypes


✅ Saved: C:\0_DA\Source\Data_Analytics_Project\data\processed\prizes.csv
Shape: (682, 8)


prize_id                 object
award_year                int64
category                 object
category_full            object
date_awarded             object
prize_status             object
prize_amount              int64
prize_amount_adjusted     int64
dtype: object

## 2.2 Create `laureates.csv` (Laureate-level table) <a id="create-laureates"></a>

**Goal:** Build a clean laureate-level table with **one row per laureate** (persons + organizations).

**Input:**
- `laureates` (loaded from the latest `data/raw/laureates_*.json`)

**Output file:**
- `data/processed/laureates.csv`

**Key fields extracted:**
- `laureate_id` (unique ID)
- `laureate_type` (person / organization)
- Names (`known_name`, `full_name`, `org_name`)
- `gender` (for persons, if available)
- Birth / death details (date, city, country) where available
- External references (`wikipedia_en`, `wikidata_id`) where available

**Actions in the next code cell:**
- Extract relevant fields from each laureate JSON record (robust for missing values)
- Normalize multilingual text fields (use English when present)
- Remove duplicates (by `laureate_id`)
- Save the cleaned table to CSV

**Validation / checks:**
- `laureates_df.shape`, `laureates_df.dtypes`
- `laureates_df["laureate_id"].isna().sum()` should be 0
- Quick preview of persons vs organizations (`value_counts`)


In [9]:
# Step 2.2 — Create laureates.csv

def pick_en(x):
    """Return English text from dict like {'en': 'Physics'}; otherwise return x."""
    if isinstance(x, dict):
        return x.get("en") or x.get("se") or next(iter(x.values()), None)
    return x

def safe_get(d, path, default=None):
    """Safely navigate nested dicts. path = ['birth','place','countryNow'] etc."""
    cur = d
    for key in path:
        if isinstance(cur, dict) and key in cur:
            cur = cur[key]
        else:
            return default
    return cur

rows = []

for l in laureates:
    lid = safe_get(l, ["id"])

    known_name = pick_en(safe_get(l, ["knownName"]))
    full_name  = pick_en(safe_get(l, ["fullName"]))
    org_name   = pick_en(safe_get(l, ["orgName"]))
    gender     = safe_get(l, ["gender"])

    # birth info (persons typically)
    birth_date = safe_get(l, ["birth", "date"])
    birth_city = pick_en(safe_get(l, ["birth", "place", "city"]))
    birth_country_now = pick_en(safe_get(l, ["birth", "place", "countryNow"]))
    birth_country_original = pick_en(safe_get(l, ["birth", "place", "country"]))

    # death info (persons typically)
    death_date = safe_get(l, ["death", "date"])
    death_city = pick_en(safe_get(l, ["death", "place", "city"]))
    death_country_now = pick_en(safe_get(l, ["death", "place", "countryNow"]))

    # external refs (not always present)
    wikipedia_en = safe_get(l, ["wikipedia", "english"])
    wikidata_id  = safe_get(l, ["wikidata", "id"])

    laureate_type = "organization" if org_name else "person"

    rows.append({
        "laureate_id": lid,
        "laureate_type": laureate_type,
        "known_name": known_name,
        "full_name": full_name,
        "org_name": org_name,
        "gender": gender,
        "birth_date": birth_date,
        "birth_city": birth_city,
        "birth_country_now": birth_country_now,
        "birth_country_original": birth_country_original,
        "death_date": death_date,
        "death_city": death_city,
        "death_country_now": death_country_now,
        "wikipedia_en": wikipedia_en,
        "wikidata_id": wikidata_id,
    })

laureates_df = (
    pd.DataFrame(rows)
    .dropna(subset=["laureate_id"])
    .drop_duplicates(subset=["laureate_id"])
    .reset_index(drop=True)
)

# Save to processed
laureates_csv_path = PROCESSED_DIR / "laureates.csv"
laureates_df.to_csv(laureates_csv_path, index=False)

print("✅ Saved:", laureates_csv_path)
print("Shape:", laureates_df.shape)
laureates_df.head(5)

✅ Saved: C:\0_DA\Source\Data_Analytics_Project\data\processed\laureates.csv
Shape: (1018, 15)


,laureate_id,laureate_type,known_name,full_name,org_name,gender,birth_date,birth_city,birth_country_now,birth_country_original,death_date,death_city,death_country_now,wikipedia_en,wikidata_id
0,745,person,A. Michael Spence,A. Michael Spence,None,male,1943-00-00,"Montclair, NJ",USA,USA,None,None,None,https://en.wikipedia.org/wiki/Michael_Spence,Q157245
1,102,person,Aage N. Bohr,Aage Niels Bohr,None,male,1922-06-19,Copenhagen,Denmark,Denmark,2009-09-08,Copenhagen,Denmark,https://en.wikipedia.org/wiki/Aage_Bohr,Q103854
2,779,person,Aaron Ciechanover,Aaron Ciechanover,None,male,1947-10-01,Haifa,Israel,British Protectorate of Palestine,None,None,None,https://en.wikipedia.org/wiki/Aaron_Ciechanover,Q233205
3,259,person,Aaron Klug,Aaron Klug,None,male,1926-08-11,Zelvas,Lithuania,Lithuania,2018-11-20,None,None,https://en.wikipedia.org/wiki/Aaron_Klug,Q190626
4,1004,person,Abdulrazak Gurnah,Abdulrazak Gurnah,None,male,1948-00-00,None,None,None,None,None,None,https://en.wikipedia.org/wiki/Abdulrazak_Gurnah,Q317877


In [8]:
print("Missing laureate_id:", laureates_df["laureate_id"].isna().sum())
print("Type counts:\n", laureates_df["laureate_type"].value_counts(dropna=False))
laureates_df.dtypes


Missing laureate_id: 0
Type counts:
 laureate_type
person          990
organization     28
Name: count, dtype: int64


laureate_id               object
laureate_type             object
known_name                object
full_name                 object
org_name                  object
gender                    object
birth_date                object
birth_city                object
birth_country_now         object
birth_country_original    object
death_date                object
death_city                object
death_country_now         object
wikipedia_en              object
wikidata_id               object
dtype: object

In [10]:
# Step 2.3 — Create prize_laureate.csv (minimal bridge table)

def pick_en(x):
    if isinstance(x, dict):
        return x.get("en") or x.get("se") or next(iter(x.values()), None)
    return x

bridge_rows = []

for p in nobel_prizes:
    award_year = p.get("awardYear")
    category = pick_en(p.get("category"))
    category_norm = str(category).strip().lower().replace(" ", "_") if category else None

    if not award_year or not category_norm:
        continue

    prize_id = f"{award_year}_{category_norm}"

    for l in (p.get("laureates") or []):
        bridge_rows.append({
            "prize_id": prize_id,
            "laureate_id": l.get("id"),
            "motivation": pick_en(l.get("motivation")),
            "share": l.get("share"),
            "portion": l.get("portion"),
            "sort_order": l.get("sortOrder"),
        })

prize_laureate_df = (
    pd.DataFrame(bridge_rows)
    .dropna(subset=["prize_id", "laureate_id"])
    .drop_duplicates(subset=["prize_id", "laureate_id", "motivation", "share", "portion"])
    .sort_values(["prize_id", "sort_order"], na_position="last")
    .reset_index(drop=True)
)

bridge_csv_path = PROCESSED_DIR / "prize_laureate.csv"
prize_laureate_df.to_csv(bridge_csv_path, index=False)

print("✅ Saved:", bridge_csv_path)
print("Shape:", prize_laureate_df.shape)
prize_laureate_df.head()

✅ Saved: C:\0_DA\Source\Data_Analytics_Project\data\processed\prize_laureate.csv
Shape: (1026, 6)


,prize_id,laureate_id,motivation,share,portion,sort_order
0,1901_chemistry,160,in recognition of the extraordinary services h...,None,1,1
1,1901_literature,569,in special recognition of his poetic compositi...,None,1,1
2,1901_peace,462,for his humanitarian efforts to help wounded s...,None,1/2,1
3,1901_peace,463,for his lifelong work for international peace ...,None,1/2,2
4,1901_physics,1,in recognition of the extraordinary services h...,None,1,1


In [11]:
prize_laureate_df.dtypes

prize_id       object
laureate_id    object
motivation     object
share          object
portion        object
sort_order     object
dtype: object

In [12]:
print("Missing prize_id:", prize_laureate_df["prize_id"].isna().sum())
print("Missing laureate_id:", prize_laureate_df["laureate_id"].isna().sum())

# Check join coverage with prizes
missing_prize_ids = set(prize_laureate_df["prize_id"]) - set(prizes["prize_id"])
print("Bridge rows with prize_id not found in prizes table:", len(missing_prize_ids))

# Check join coverage with laureates
missing_laureate_ids = set(prize_laureate_df["laureate_id"].astype(str)) - set(laureates_df["laureate_id"].astype(str))
print("Bridge rows with laureate_id not found in laureates table:", len(missing_laureate_ids))


Missing prize_id: 0
Missing laureate_id: 0
Bridge rows with prize_id not found in prizes table: 0
Bridge rows with laureate_id not found in laureates table: 0


## 2.4 Build `analysis_dataset` (Join tables for EDA & Dashboard) <a id="build-analysis-dataset"></a>

**Goal:** Create a single analysis-ready dataset by joining the three processed tables:
- `prizes` (prize-level)
- `laureates` (laureate-level)
- `prize_laureate` (bridge table)

**Inputs (processed files):**
- `data/processed/prizes.csv`
- `data/processed/laureates.csv`
- `data/processed/prize_laureate.csv`

**Output file:**
- `data/processed/analysis_dataset.analysis_dataset.csv`)*

**Actions in the next code cell:**
- Load the 3 CSV files
- Join:
  - `prize_laureate` + `prizes` on `prize_id`
  - result + `laureates` on `laureate_id`
- Create helpful derived columns:
  - `decade`
  - `laureate_type` (person/organization)
  - parsed date columns (optional)
- Run validation checks (row counts, missing join keys)
- Save final dataset for EDA and Dash


In [13]:
# Step 2.4 — Build analysis dataset by joining the 3 tables

# 1) Load processed tables
prizes = pd.read_csv(PROCESSED_DIR / "prizes.csv")
laureates_df = pd.read_csv(PROCESSED_DIR / "laureates.csv")
prize_laureate_df = pd.read_csv(PROCESSED_DIR / "prize_laureate.csv")

print("Loaded:")
print(" - prizes:", prizes.shape)
print(" - laureates:", laureates_df.shape)
print(" - prize_laureate:", prize_laureate_df.shape)

# 2) Ensure join keys are consistent types (strings are safest for IDs)
prizes["prize_id"] = prizes["prize_id"].astype("string")
prize_laureate_df["prize_id"] = prize_laureate_df["prize_id"].astype("string")

laureates_df["laureate_id"] = laureates_df["laureate_id"].astype("string")
prize_laureate_df["laureate_id"] = prize_laureate_df["laureate_id"].astype("string")

# 3) Join bridge -> prizes
df = prize_laureate_df.merge(
    prizes,
    on="prize_id",
    how="left",
    suffixes=("", "_prize")
)

# 4) Join result -> laureates
df = df.merge(
    laureates_df,
    on="laureate_id",
    how="left",
    suffixes=("", "_laureate")
)

# 5) Derived features (helpful for EDA & dashboard)
df["award_year"] = pd.to_numeric(df["award_year"], errors="coerce")
df["decade"] = (df["award_year"] // 10) * 10

# Optional: parse dates (safe)
df["birth_date_parsed"] = pd.to_datetime(df.get("birth_date"), errors="coerce")
df["date_awarded_parsed"] = pd.to_datetime(df.get("date_awarded"), errors="coerce")

# Age at award (persons only; if birth date exists)
df["age_at_award"] = (df["award_year"] - df["birth_date_parsed"].dt.year).astype("Float64")
df.loc[df["laureate_type"] != "person", "age_at_award"] = pd.NA

# 6) Validation checks
missing_prize_join = df["category_full"].isna().sum()
missing_laureate_join = df["laureate_type"].isna().sum()

print("\nJoin validation:")
print(" - rows in final df:", df.shape[0])
print(" - missing prize join (category_full NA):", missing_prize_join)
print(" - missing laureate join (laureate_type NA):", missing_laureate_join)
print(" - max award year:", int(df["award_year"].max()))


Loaded:
 - prizes: (682, 8)
 - laureates: (1018, 15)
 - prize_laureate: (1026, 6)

Join validation:
 - rows in final df: 1026
 - missing prize join (category_full NA): 0
 - missing laureate join (laureate_type NA): 0
 - max award year: 2025


In [14]:
# Save final dataset as CSV (no parquet needed)
analysis_csv_path = PROCESSED_DIR / "analysis_dataset.csv"
df.to_csv(analysis_csv_path, index=False)

print("✅ Saved:", analysis_csv_path)


✅ Saved: C:\0_DA\Source\Data_Analytics_Project\data\processed\analysis_dataset.csv


In [17]:
analysis_dataset_df = pd.read_csv(PROCESSED_DIR / "analysis_dataset.csv")

In [18]:
analysis_dataset_df.dtypes

prize_id                   object
laureate_id                 int64
motivation                 object
share                     float64
portion                    object
sort_order                  int64
award_year                  int64
category                   object
category_full              object
date_awarded               object
prize_status              float64
prize_amount                int64
prize_amount_adjusted       int64
laureate_type              object
known_name                 object
full_name                  object
org_name                   object
gender                     object
birth_date                 object
birth_city                 object
birth_country_now          object
birth_country_original     object
death_date                 object
death_city                 object
death_country_now          object
wikipedia_en               object
wikidata_id                object
decade                      int64
birth_date_parsed          object
date_awarded_p